## Imports

In [1]:
import torch
import torchtext
import pytorch_lightning as pl

from data import TEXT, LABEL, test_data
import models

Loading data...
Loading vocabulary...


## Loading a model

In [2]:
MODEL = None

# Load a model from checkpoint and use it for inference
def load_model(encoder, version = None):
    # If no version specified, use the latest model
    if version is None:
        versions = [int(f[8:-5]) for f in os.listdir(f"checkpoints/{encoder}")]
        version = versions[-1]

    model_path = f"checkpoints/{encoder}/version_{version}.ckpt"
    print("Using " + model_path)
    global MODEL
    MODEL = models.Classifier.load_from_checkpoint(model_path).eval()

load_model("pooledbilstm")

Using checkpoints/pooledbilstm/version_1.ckpt


# Interactive Inference

In [3]:
# Print batch of token indices as text
def print_text(tokens):
    if len(tokens.shape) == 1: tokens = [tokens]
    for s in tokens:
        print(" ".join([TEXT.vocab.itos[t] if t > 1 else '' for t in s]))

# Make entailment prediction from premise and hypothesis
def predict(premise, hypothesis):
    print(f"\x1b[0m\"{premise}\" -> \"{hypothesis}\":")
    premise = TEXT.process([TEXT.preprocess(premise)])
    hypothesis = TEXT.process([TEXT.preprocess(hypothesis)])
    MODEL.eval()
    preds = MODEL(premise, hypothesis).softmax(1)
    pred = preds.argmax()

    color = '\x1b[0m'
    if pred == 0: color = '\x1b[1;32m' # entailment
    elif pred == 1: color = '\x1b[1;31m' # contradiction
    elif pred == 2: color = '\x1b[1;33m' # neutral

    print(f"    {color}{LABEL.vocab.itos[pred]} ({preds[0, pred] * 100:.2f}%)\n")

In [4]:
predict("Alice is going for a walk.", "Alice is cleaning her room.")
predict("Alice is going for a walk.", "Alice is wearing a hat.")
predict("Alice is going for a walk.", "Alice is outside.")

"Alice is going for a walk." -> "Alice is cleaning her room.":
    contradiction (98.93%)

"Alice is going for a walk." -> "Alice is wearing a hat.":
    neutral (96.08%)

"Alice is going for a walk." -> "Alice is outside.":
    entailment (81.85%)



# Analysis

In [20]:
from tqdm import tqdm

# Get predictions for each item in test set
percentages = []
for e in tqdm(test_data):
    premise = TEXT.process([e.premise])
    hypothesis = TEXT.process([e.hypothesis])
    label = LABEL.process([e.label]).squeeze()
    preds = MODEL(premise, hypothesis).softmax(1).squeeze()
    percentages.append(preds[label].item())

# Sort datapoints by how correct the model was
difficulty_ranking = sorted(range(len(percentages)), key=lambda k: percentages[k])

100%|██████████| 9824/9824 [02:08<00:00, 76.22it/s]


In [47]:
# Easiest examples
for i in list(reversed(difficulty_ranking))[:10]:
    premise = TEXT.process([test_data[i].premise])
    hypothesis = TEXT.process([test_data[i].hypothesis])
    label = LABEL.process([test_data[i].label]).squeeze()
    preds = MODEL(premise, hypothesis).softmax(1).squeeze()
    pred = preds.argmax()
    print_text(premise)
    print_text(hypothesis)
    print(f"Correct: {test_data[i].label}")
    print(f"Prediction: {LABEL.vocab.itos[pred]} ({preds[pred] * 100:.2f}%)\n"

A dog standing near snow looking at water .
A cat is laying on the couch .
Correct: contradiction
Prediction: contradiction (100.00%)

A brown dog and a black and white dog are running over grassy hills .
one cat sleeps
Correct: contradiction
Prediction: contradiction (100.00%)

An olympian leaps over an obstacle .
An old woman sleeps in the kitchen .
Correct: contradiction
Prediction: contradiction (100.00%)

A child dressed in purple clothing and a hat runs in splashing water while a man and a small boy walk nearby .
a cat sleeps on the couch
Correct: contradiction
Prediction: contradiction (100.00%)

A black dog is digging in the snow .
A cat running down a beach
Correct: contradiction
Prediction: contradiction (100.00%)

Young child standing on a snow pile in a city .
the man is at home sleeping
Correct: contradiction
Prediction: contradiction (100.00%)

Woman running in a park while listening to music .
A man cooking at home .
Correct: contradiction
Prediction: contradiction (100.

In [48]:
# Most difficult examples
for i in difficulty_ranking[:10]:
    premise = TEXT.process([test_data[i].premise])
    hypothesis = TEXT.process([test_data[i].hypothesis])
    label = LABEL.process([test_data[i].label]).squeeze()
    preds = MODEL(premise, hypothesis).softmax(1).squeeze()
    pred = preds.argmax()
    print_text(premise)
    print_text(hypothesis)
    print(f"Correct: {test_data[i].label}")
    print(f"Prediction: {LABEL.vocab.itos[pred]} ({preds[pred] * 100:.2f}%)\n")

Several women are playing volleyball .
This does n't look like soccer
Correct: entailment
Prediction: contradiction (99.94%)

A man wearing a welding mask working on a duct .
The man is working underneath a vehicle .
Correct: contradiction
Prediction: entailment (99.14%)

An older gentleman looks at the camera while he is building a deck .
An older gentleman in overalls looks at the camera while he is building a stained red deck in front of a house .
Correct: entailment
Prediction: neutral (98.63%)

A teenage girl in a crowd of people looking up at something .
no one saw the rainbow in the sky
Correct: neutral
Prediction: contradiction (99.92%)

A group of girls jumping over another girl who is laying on the floor .
A gorup of girl is laying down .
Correct: contradiction
Prediction: entailment (99.12%)

People are riding bicycles in the street , and they are all wearing helmets .
A group of friends are grabbing their bikes , getting ready for the morning bike ride
Correct: contradictio